In [1]:
import pandas as pd
import numpy as np

import time
import os
import copy
import pickle
from collections import deque

# deep learning imports
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import DataLoader

import warnings
warnings.filterwarnings("ignore")

/home/paperspace/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
directory = "./../../.local/bin/dc-grocery-outlet-models/image_splits_10_or_less"


def list_files(d):
    """
        just lists files given a directory (str) d
    """
    r = []
    for root, dirs, files in os.walk(d):
        for name in files:
            r.append(os.path.join(root, name))
    return r


In [3]:
files = list_files(directory)
print(f"there are {len(files)} files")

there are 50 files


In [4]:
def read_make_image_splits(file):
    """
        given a .pkl file, returns a 3D numpy array
    """

    pkl_file = open(file, 'rb')

    data1 = pickle.load(pkl_file)
    pkl_file.close()

    return data1

def preprocess(file):
    """
        given a file, returns generators for training, test, and validation
    """
    
    df = pd.DataFrame(read_make_image_splits(file), columns=["brand", "size", "measure", "class", "long_description", "ImageList", "image_matrix", "image_tensor"])
    df = df.dropna()
    df_brand = df[["image_tensor", "brand"]]
    
    # to be replaced
    train = list(map(list, zip(df_brand["image_tensor"].values, df_brand["brand"].values)))
    
    random.shuffle(train)
    val_split = int(len(train)*0.8)
    test_split = int(len(train)*0.9)
    val = train[val_split:test_split]
    test = train[test_split:]
    train = train[:val_split]
    # print(train[0])
    # print(train[0][0])
    # train, val = train_test_split(train, test_size=0.2)
    # train, test = train_test_split(train, test_size=0.1)

    train_generator = data.DataLoader(train, batch_size=16)
    test_generator = data.DataLoader(test, batch_size=16)
    val_generator = data.DataLoader(val, batch_size=16)
    
    return train_generator, test_generator, val_generator, val

In [5]:
f1 = pd.DataFrame(read_make_image_splits(files[0]), 
                  columns=["brand", "size", "measure", "class", "long_description", "ImageList", "image_matrix", "image_tensor"])
f1 = f1.dropna()
f2 = pd.DataFrame(read_make_image_splits(files[1]), 
                  columns=["brand", "size", "measure", "class", "long_description", "ImageList", "image_matrix", "image_tensor"])
f2 = f2.dropna()

In [6]:
f1.iloc[0]

brand                                       made good                
size                                                                6
measure                                                       Count  
class                                                              56
long_description             MADE GOOD ORGANIC CRISPY SQUARES CARAMEL
ImageList                                     prod/00687456213415.jpg
image_matrix        [[[255, 255, 255], [255, 255, 255], [255, 255,...
image_tensor        [[[tensor(255, dtype=torch.uint8), tensor(255,...
Name: 0, dtype: object

In [7]:
#f1brands = set(f1['brand'].unique())
#f2brands = set(f2['brand'].unique())
#len(list(f1brands - f2brands))

In [8]:
#np.sum(f1.brand.isin(f2brands)) / len(f1brands)

In [9]:
class GroceryDataset(torch.utils.data.Dataset):

    def __init__(self, data, transform=None):
        super(GroceryDataset, self).__init__()
        np.random.seed(42)
        self.transform = transform
        self.data = data # f1
    
    def __len__(self):
        return len(self.data)
        
    def __getitem__(self, index):
        # image1 = random.choice(self.dataset.imgs)
        '''
            Gets two images from dataframe from either the same or different classes randomly 
            with the appropriate label (0 = different brand, 1 = same brand)
        '''
        x = np.random.uniform()
        sample = self.data.sample(1)
        image1 = sample['image_tensor'].iloc[0]
        brand1 = sample['brand'].iloc[0]
        if (x >= 0.9):
            sample2 = self.data[self.data['brand'] == brand1].sample(1)
            label = 1
        else:
            sample2 = self.data[self.data['brand'] != brand1].sample(1)
            label = 0
        image2 = sample2['image_tensor'].iloc[0]
        brand2 = sample2['brand'].iloc[0]
        if self.transform:
            image1 = self.transform(image1)
            image2 = self.transform(image2)
        return image1, image2, torch.from_numpy(np.array([label], dtype=np.float32))

In [10]:
class Siamese(nn.Module):

    def __init__(self):
        super(Siamese, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3, 64, 10),  # 64@96*96
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),  # 64@48*48
            nn.Conv2d(64, 64, 7),
            nn.ReLU(),    # 128@42*42
            nn.MaxPool2d(2),   # 128@21*21
            nn.Conv2d(64, 32, 4),
            nn.ReLU(), # 128@18*18
            nn.MaxPool2d(2), # 128@9*9
            nn.Conv2d(32, 32, 4),
            nn.ReLU(),   # 256@6*6
        )
        self.linear = nn.Sequential(nn.Linear(9216, 4096), nn.Sigmoid())
        self.out = nn.Linear(4096, 1)

    def forward_one(self, x):
        x = self.conv(x)
        x = x.view(x.size()[0], -1)
        x = self.linear(x)
        return x

    def forward(self, x1, x2):
        out1 = self.forward_one(x1)
        out2 = self.forward_one(x2)
        dis = torch.abs(out1 - out2)
        out = self.out(dis)
        #  return self.sigmoid(out)
        return out

In [11]:
data_transforms = transforms.Compose([transforms.Resize((224, 224))])

In [12]:
Flags = {"batch_size" : 128, "lr" : 0.00006, "show_every" : 10, "save_every" : 100, "test_every" : 100, "max_iter" : 50000, "model_path" : "./siamese",
         "gpu_ids" : "0, 1, 2, 3", "workers" : 4, "times" : 400, "way" : 20, "cuda" : True}

In [13]:
loss_fn = torch.nn.BCEWithLogitsLoss(size_average=True)
net = Siamese()
net.cuda()
net.train()

trainSet = GroceryDataset(f1, transform=data_transforms)
valSet = GroceryDataset(f2, transform=data_transforms)

trainLoader = DataLoader(trainSet, batch_size=Flags['way'], shuffle=False, num_workers=2)
valLoader = DataLoader(valSet, batch_size=Flags['batch_size'], shuffle=False, num_workers=2)

optimizer = torch.optim.Adam(net.parameters(), lr = Flags['lr'] )
optimizer.zero_grad()

In [14]:
train_loss = []
loss_val = 0
time_start = time.time()
queue = deque(maxlen=20)

# input parameters to the model

for batch_id, (img1, img2, label) in enumerate(trainLoader, 1): # this is just one-indexing
    if batch_id > Flags['max_iter']:
        break
    img1, img2, label = img1.type(torch.FloatTensor).cuda(), img2.type(torch.FloatTensor).cuda(), label.cuda()
    optimizer.zero_grad()
    output = net.forward(img1, img2)
    loss = loss_fn(output, label)
    loss_val += loss.item()
    loss.backward()
    optimizer.step()
    if batch_id % Flags['show_every'] == 0 :
        print('[%d]\tloss:\t%.5f\ttime lapsed:\t%.2f s'%(batch_id, loss_val/Flags['show_every'], time.time() - time_start))
        loss_val = 0
        time_start = time.time()
    if batch_id % Flags['save_every'] == 0:
        torch.save(net.state_dict(), Flags['model_path'] + '/model-inter-' + str(batch_id+1) + ".pt")
    if batch_id % Flags['test_every'] == 0:
        right, error = 0, 0
        for _, (test1, test2) in enumerate(testLoader, 1):
            if Flags['cuda']:
                test1, test2 = test1.cuda(), test2.cuda()
            test1, test2 = Variable(test1), Variable(test2)
            output = net.forward(test1, test2).data.cpu().numpy()
            pred = np.argmax(output)
            if pred == 0:
                right += 1
            else: error += 1
        print('*'*70)
        print('[%d]\tTest set\tcorrect:\t%d\terror:\t%d\tprecision:\t%f'%(batch_id, right, error, right*1.0/(right+error)))
        print('*'*70)
        queue.append(right*1.0/(right+error))
    train_loss.append(loss_val)
#  learning_rate = learning_rate * 0.95

with open('train_loss', 'wb') as f:
    pickle.dump(train_loss, f)

acc = 0.0
for d in queue:
    acc += d
print("#"*70)
print("final accuracy: ", acc/20)

RuntimeError: CUDA out of memory. Tried to allocate 56.00 MiB (GPU 0; 15.74 GiB total capacity; 395.06 MiB already allocated; 15.75 MiB free; 396.00 MiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF